In [ ]:
'''Exports a trained model into a frozen graph'''

In [1]:
import argparse
import glob
import os
import sys
import time

import PIL
#import humanfriendly
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import tensorflow as tf
from tqdm import tqdm

from tensorflow.python.tools import freeze_graph


In [10]:
sess = tf.Session()

In [8]:
path_to_outputs = '../../../../outputs/fasterRCNN_07_02_img_resize'

pbtxt_path = os.path.join(path_to_outputs, 'graph.pbtxt')
ckpt_path = os.path.join(path_to_outputs, 'model.ckpt-300000')
frozen_graph_path = os.path.join(path_to_outputs, "frozen_graph_test.pb")

In [ ]:
freeze_graph.freeze_graph(input_graph = pbtxt_filepath,
                              input_saver = '',  input_binary = False, 
                              input_checkpoint = ckpt_filepath, 
                              output_node_names = 'group_deps', 
                              restore_op_name = '', 
                              filename_tensor_name = '',
                              output_graph = pb_filepath, 
                              clear_devices = True,  initializer_nodes = '')

WHY WORKS IN THE OTHER INFERENCE_TEST NOTEBOOK????

In [11]:
freeze_graph.freeze_graph(input_graph=pbtxt_path,
                          input_saver='',input_binary=False,
                          input_checkpoint=ckpt_path,
                          #output_node_names='Merge/MergeSummary', -_> Dst tensor is not initialized.
                          output_node_names = 'group_deps', 
                          #output_node_names ? Found the last node name in pbtxt_file
                          
                          restore_op_name='save/restore_all',
                          filename_tensor_name='save/Const:0',
                          output_graph=frozen_graph_path,
                          clear_devices=True,initializer_nodes='')


INFO:tensorflow:Restoring parameters from ../../../../outputs/fasterRCNN_07_02_img_resize/model.ckpt-300000


InternalError: Dst tensor is not initialized.
	 [[Node: save_2/RestoreV2/_725 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_730_save_2/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: save_2/RestoreV2/_194 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_200_save_2/RestoreV2", _device="/job:localhost/replica:0/task:0/device:CPU:0"](save_2/RestoreV2:97)]]

# Aternatives

### Siyu's method

In [ ]:
def load_model(checkpoint):
    """
    Load a detection model (i.e., create a graph) from a .pb file
    """

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(checkpoint, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    
    return detection_graph

ckpt_path = "../../../outputs/train_06-06/model.ckpt-300000"
load_model(ckpt_path)